# Part 4: Disaster Strikes

In [2]:
#q8
!hdfs dfsadmin -fs hdfs://boss:9000 -report

Configured Capacity: 51642105856 (48.10 GB)
Present Capacity: 27796188284 (25.89 GB)
DFS Remaining: 27267203072 (25.39 GB)
DFS Used: 528985212 (504.48 MB)
DFS Used%: 1.90%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (2):

Name: 172.25.0.2:9866 (172.25.0.2)
Hostname: 06ac5bfd2283
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 267049841 (254.68 MB)
Non DFS Used: 11903603855 (11.09 GB)
DFS Remaining: 13633622016 (12.70 GB)
DFS Used%: 1.03%
DFS Remaining%: 52.80%
Configured Cache Capacity:

In [4]:
import time
time.sleep(30)

In [3]:
#q9
import requests
r = requests.get(f"http://boss:9870/webhdfs/v1/single.csv?op=GETFILEBLOCKLOCATIONS")
containerDict = {}
for i in range(len(r.json().get("BlockLocations").get("BlockLocation"))):
    offset = r.json().get("BlockLocations").get("BlockLocation")[i].get("offset")
    status = requests.get(f"http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset={offset}&noredirect=true");
    if (status.status_code == 403):
        id = "lost"
    else:
        id = r.json().get("BlockLocations").get("BlockLocation")[i].get("hosts")[0]
    blocks = r.json().get("BlockLocations").get("BlockLocation")[i].get("length") / 1048576
    count = containerDict.setdefault(id, 0)
    if (count == None):
            count = 0
    containerDict.update({id: (count+round(blocks))});
containerDict

{'06ac5bfd2283': 86, '11df0cb46c12': 81}

In [4]:
import io
import re

In [5]:
#q10
import requests
count = 0
r = requests.get(f"http://boss:9870/webhdfs/v1/single.csv?op=GETFILEBLOCKLOCATIONS")
for i in range(len(r.json().get("BlockLocations").get("BlockLocation"))):
    offset = r.json().get("BlockLocations").get("BlockLocation")[i].get("offset")
    length = r.json().get("BlockLocations").get("BlockLocation")[i].get("length")
    status = requests.get(f"http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset={offset}&length={length}");
    # print(status)
    if (status.status_code != 403):
        content = io.BytesIO(status.content)
        reader = io.TextIOWrapper(io.BufferedReader(content))
        for i, line in enumerate(reader):
            if re.search(r'Single Family', line):
                count += 1
        # with status.content as f:
        #     reader = io.TextIOWrapper(io.BufferedReader(f))
        #     for i, line in enumerate(reader):
        #         if re.search(r'Single Family', line):
        #             count += 1
count

ConnectionError: HTTPConnectionPool(host='06ac5bfd2283', port=9864): Max retries exceeded with url: /webhdfs/v1/single.csv?op=OPEN&namenoderpcaddress=boss:9000&length=1048576&offset=0 (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x7f1af434cf10>: Failed to resolve '06ac5bfd2283' ([Errno -2] Name or service not known)"))